# Credit Risk Resampling Techniques

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

# Read the CSV and Perform Basic Data Cleaning

In [3]:
columns = [
    "loan_amnt", "int_rate", "installment", "home_ownership",
    "annual_inc", "verification_status", "issue_d", "loan_status",
    "pymnt_plan", "dti", "delinq_2yrs", "inq_last_6mths",
    "open_acc", "pub_rec", "revol_bal", "total_acc",
    "initial_list_status", "out_prncp", "out_prncp_inv", "total_pymnt",
    "total_pymnt_inv", "total_rec_prncp", "total_rec_int", "total_rec_late_fee",
    "recoveries", "collection_recovery_fee", "last_pymnt_amnt", "next_pymnt_d",
    "collections_12_mths_ex_med", "policy_code", "application_type", "acc_now_delinq",
    "tot_coll_amt", "tot_cur_bal", "open_acc_6m", "open_act_il",
    "open_il_12m", "open_il_24m", "mths_since_rcnt_il", "total_bal_il",
    "il_util", "open_rv_12m", "open_rv_24m", "max_bal_bc",
    "all_util", "total_rev_hi_lim", "inq_fi", "total_cu_tl",
    "inq_last_12m", "acc_open_past_24mths", "avg_cur_bal", "bc_open_to_buy",
    "bc_util", "chargeoff_within_12_mths", "delinq_amnt", "mo_sin_old_il_acct",
    "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op", "mo_sin_rcnt_tl", "mort_acc",
    "mths_since_recent_bc", "mths_since_recent_inq", "num_accts_ever_120_pd", "num_actv_bc_tl",
    "num_actv_rev_tl", "num_bc_sats", "num_bc_tl", "num_il_tl",
    "num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0",
    "num_sats", "num_tl_120dpd_2m", "num_tl_30dpd", "num_tl_90g_dpd_24m",
    "num_tl_op_past_12m", "pct_tl_nvr_dlq", "percent_bc_gt_75", "pub_rec_bankruptcies",
    "tax_liens", "tot_hi_cred_lim", "total_bal_ex_mort", "total_bc_limit",
    "total_il_high_credit_limit", "hardship_flag", "debt_settlement_flag"
]

target = ["loan_status"]

In [4]:
# Load the data
file_path = Path('../LoanStats_2019Q1.csv')
df = pd.read_csv(file_path, skiprows=1)[:-2]
df = df.loc[:, columns].copy()

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

# Remove the `Issued` loan status
issued_mask = df['loan_status'] != 'Issued'
df = df.loc[issued_mask]

# convert interest rate to numerical
df['int_rate'] = df['int_rate'].str.replace('%', '')
df['int_rate'] = df['int_rate'].astype('float') / 100


# Convert the target column values to low_risk and high_risk based on their values
x = {'Current': 'low_risk'}   
df = df.replace(x)

x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period'], 'high_risk')    
df = df.replace(x)

# convert N to 0 and Y to Yes
x = dict.fromkeys(['n','N'],0)
df = df.replace(x)
x = dict.fromkeys(['y','Y'],0)
df = df.replace(x)

# dictionary for home ownership
ownership_num = {
    'MORTGAGE': 0,
    'RENT': 1,
    'OWN': 2,
    'ANY': 3
}

df["home_ownership_int"] = df["home_ownership"].apply(lambda x: ownership_num[x])
df = df.drop("home_ownership",axis=1)

# dictionary for verification
verification_num = {
    'Not Verified': 0,
    'Source Verified': 1,
    'Verified': 2
}

df["verification_status_int"] = df["verification_status"].apply(lambda x: verification_num[x])
df = df.drop("verification_status",axis=1)

# dictionary for issue date
month_num = {
    'Jan-2019': 0,
    'Feb-2019': 1,
    'Mar-2019': 2,
    'Apr-2019': 3,
    'May-2019': 4,
}

df["issue_d_int"] = df["issue_d"].apply(lambda x: month_num[x])
df = df.drop("issue_d",axis=1)

df["next_pymnt_d_int"] = df["next_pymnt_d"].apply(lambda x: month_num[x])
df = df.drop("next_pymnt_d",axis=1)

# encode for application type and list status
df = pd.get_dummies(df, columns=["application_type","initial_list_status"])

df.reset_index(inplace=True, drop=True)

df.head()

,loan_amnt,int_rate,installment,annual_inc,loan_status,pymnt_plan,dti,delinq_2yrs,inq_last_6mths,open_acc,...,hardship_flag,debt_settlement_flag,home_ownership_int,verification_status_int,issue_d_int,next_pymnt_d_int,application_type_Individual,application_type_Joint App,initial_list_status_f,initial_list_status_w
0,10500.0,0.1719,375.35,66000.0,low_risk,0,27.24,0.0,0.0,8.0,...,0,0,1,1,2,4,1,0,0,1
1,25000.0,0.2000,929.09,105000.0,low_risk,0,20.23,0.0,0.0,17.0,...,0,0,0,2,2,4,1,0,0,1
2,20000.0,0.2000,529.88,56000.0,low_risk,0,24.26,0.0,0.0,8.0,...,0,0,0,2,2,4,1,0,0,1
3,10000.0,0.1640,353.55,92000.0,low_risk,0,31.44,0.0,1.0,10.0,...,0,0,1,2,2,4,1,0,0,1
4,22000.0,0.1474,520.39,52000.0,low_risk,0,18.76,0.0,1.0,14.0,...,0,0,0,0,2,4,1,0,0,1


In [5]:
df.sample(10)

,loan_amnt,int_rate,installment,annual_inc,loan_status,pymnt_plan,dti,delinq_2yrs,inq_last_6mths,open_acc,...,hardship_flag,debt_settlement_flag,home_ownership_int,verification_status_int,issue_d_int,next_pymnt_d_int,application_type_Individual,application_type_Joint App,initial_list_status_f,initial_list_status_w
17116,12000.0,0.2880,379.45,36000.0,low_risk,0,36.40,0.0,1.0,10.0,...,0,0,1,2,1,4,1,0,0,1
5496,6000.0,0.1474,207.23,40000.0,low_risk,0,7.89,1.0,2.0,19.0,...,0,0,1,1,2,4,1,0,0,1
9221,6000.0,0.0702,185.32,50000.0,low_risk,0,13.23,0.0,0.0,5.0,...,0,0,0,1,2,4,1,0,0,1
1854,15000.0,0.1640,367.97,63000.0,low_risk,0,19.12,0.0,1.0,10.0,...,0,0,0,0,2,4,1,0,0,1
10774,30000.0,0.1180,664.31,97000.0,low_risk,0,15.03,0.0,0.0,11.0,...,0,0,1,0,2,4,1,0,0,1
23838,7800.0,0.1992,289.56,87000.0,low_risk,0,21.83,0.0,2.0,10.0,...,0,0,0,1,1,3,1,0,0,1
44130,30000.0,0.1180,664.31,140000.0,low_risk,0,12.52,0.0,1.0,9.0,...,0,0,0,0,0,3,1,0,0,1
43855,6000.0,0.0702,185.32,37500.0,low_risk,0,5.70,0.0,0.0,12.0,...,0,0,0,0,0,4,1,0,0,1
68637,20000.0,0.0756,622.68,155000.0,low_risk,0,16.99,0.0,0.0,10.0,...,0,0,0,1,0,4,1,0,0,1
63441,9000.0,0.0756,280.21,26000.0,low_risk,0,27.33,0.0,0.0,16.0,...,0,0,1,1,0,4,1,0,0,1


In [6]:
# get info for encoding
df.select_dtypes(['object'])

,loan_status
0,low_risk
1,low_risk
2,low_risk
3,low_risk
4,low_risk
...,...
68812,low_risk
68813,low_risk
68814,low_risk
68815,low_risk


# Split the Data into Training and Testing

In [7]:
# Create our features
X = df.drop('loan_status',axis=1)

# Create our target
y = df['loan_status']

In [8]:
X.describe()

,loan_amnt,int_rate,installment,annual_inc,pymnt_plan,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,...,hardship_flag,debt_settlement_flag,home_ownership_int,verification_status_int,issue_d_int,next_pymnt_d_int,application_type_Individual,application_type_Joint App,initial_list_status_f,initial_list_status_w
count,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.0,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,...,68817.0,68817.0,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000
mean,16677.594562,0.127718,480.652863,8.821371e+04,0.0,21.778153,0.217766,0.497697,12.587340,0.126030,...,0.0,0.0,0.599009,0.669994,0.726172,3.616839,0.860340,0.139660,0.123879,0.876121
std,10277.348590,0.048130,288.062432,1.155800e+05,0.0,20.199244,0.718367,0.758122,6.022869,0.336797,...,0.0,0.0,0.713731,0.719105,0.743862,0.486161,0.346637,0.346637,0.329446,0.329446
min,1000.000000,0.060000,30.890000,4.000000e+01,0.0,0.000000,0.000000,0.000000,2.000000,0.000000,...,0.0,0.0,0.000000,0.000000,0.000000,3.000000,0.000000,0.000000,0.000000,0.000000
25%,9000.000000,0.088100,265.730000,5.000000e+04,0.0,13.890000,0.000000,0.000000,8.000000,0.000000,...,0.0,0.0,0.000000,0.000000,0.000000,3.000000,1.000000,0.000000,0.000000,1.000000
50%,15000.000000,0.118000,404.560000,7.300000e+04,0.0,19.760000,0.000000,0.000000,11.000000,0.000000,...,0.0,0.0,0.000000,1.000000,1.000000,4.000000,1.000000,0.000000,0.000000,1.000000
75%,24000.000000,0.155700,648.100000,1.040000e+05,0.0,26.660000,0.000000,1.000000,16.000000,0.000000,...,0.0,0.0,1.000000,1.000000,1.000000,4.000000,1.000000,0.000000,0.000000,1.000000
max,40000.000000,0.308400,1676.230000,8.797500e+06,0.0,999.000000,18.000000,5.000000,72.000000,4.000000,...,0.0,0.0,3.000000,2.000000,2.000000,4.000000,1.000000,1.000000,1.000000,1.000000


In [9]:
# Check the balance of our target values
y.value_counts()

low_risk     68470
high_risk      347
Name: loan_status, dtype: int64

In [10]:
# Create X_train, X_test, y_train, y_test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)
Counter(y_train)

Counter({'low_risk': 51352, 'high_risk': 260})

# Oversampling

In this section, you will compare two oversampling algorithms to determine which algorithm results in the best performance. You will oversample the data using the naive random oversampling algorithm and the SMOTE algorithm. For each algorithm, be sure to complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

### Naive Random Oversampling

In [11]:
# Resample the training data with the RandomOversampler
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=1)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)

Counter(y_resampled)

Counter({'low_risk': 51352, 'high_risk': 51352})

In [12]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=1, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [13]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score

y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.6480967805616995

In [14]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix


confusion_matrix(y_test, y_pred)

array([[   57,    30],
       [ 6145, 10973]], dtype=int64)

In [15]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.66      0.64      0.02      0.65      0.42        87
   low_risk       1.00      0.64      0.66      0.78      0.65      0.42     17118

avg / total       0.99      0.64      0.66      0.78      0.65      0.42     17205



### SMOTE Oversampling

In [16]:
# Resample the training data with SMOTE
from imblearn.over_sampling import SMOTE
X_resampled, y_resampled = SMOTE(random_state=1).fit_resample(
    X_train, y_train
)
Counter(y_resampled)

Counter({'low_risk': 51352, 'high_risk': 51352})

In [17]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=1, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [18]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score

y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.6273060017485124

In [19]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, y_pred)

array([[   52,    35],
       [ 5873, 11245]], dtype=int64)

In [20]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.60      0.66      0.02      0.63      0.39        87
   low_risk       1.00      0.66      0.60      0.79      0.63      0.39     17118

avg / total       0.99      0.66      0.60      0.79      0.63      0.39     17205



# Undersampling

In this section, you will test an undersampling algorithms to determine which algorithm results in the best performance compared to the oversampling algorithms above. You will undersample the data using the Cluster Centroids algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [21]:
# Resample the data using the ClusterCentroids resampler
from imblearn.under_sampling import ClusterCentroids
cc = ClusterCentroids(random_state=1)
X_resampled, y_resampled = cc.fit_resample(X_train, y_train)
Counter(y_resampled)

Counter({'high_risk': 260, 'low_risk': 260})

In [22]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=1, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [23]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score

y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.5215189898916648

In [24]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[   55,    32],
       [10085,  7033]], dtype=int64)

In [25]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.63      0.41      0.01      0.51      0.27        87
   low_risk       1.00      0.41      0.63      0.58      0.51      0.25     17118

avg / total       0.99      0.41      0.63      0.58      0.51      0.25     17205



# Combination (Over and Under) Sampling

In this section, you will test a combination over- and under-sampling algorithm to determine if the algorithm results in the best performance compared to the other sampling algorithms above. You will resample the data using the SMOTEENN algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [26]:
# Resample the training data with SMOTEENN
from imblearn.combine import SMOTEENN
smote_enn = SMOTEENN(random_state=1)
X_resampled, y_resampled = smote_enn.fit_resample(X, y)
Counter(y_resampled)

Counter({'high_risk': 68458, 'low_risk': 62022})

In [27]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=1, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [28]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score

y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.6373418180499655

In [29]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[  62,   25],
       [7497, 9621]], dtype=int64)

In [30]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.71      0.56      0.02      0.63      0.41        87
   low_risk       1.00      0.56      0.71      0.72      0.63      0.39     17118

avg / total       0.99      0.56      0.71      0.72      0.63      0.39     17205

